# 挖掘稀疏性

在前面的章节中，我们发现要进行最大后验推断需要不断地求解大规模稀疏线性系统，即
$$\Delta^{*} = argmin_{\Delta}\sum_i ||A_i\Delta_i -b_i||_2^2$$
每一项(term)都是从原始非线性因子通过线性化得到。

## 一、雅各比矩阵稀疏性
在线性化后，我们会得到一个稀疏系统[A|b]，如下图所示。将[A|b]与因子图比较可以看出，首先每个因子节点对应一个block row，每个变量节点对应一个block column。
<div align="center"><img src="../resources/chapter3/jacobian_sparsity.png" width=50%></div>

## 二、Hessian矩阵稀疏性
由于Jacobian矩阵A具有稀疏性，则Hessian矩阵$A^TA$也具有稀疏性。具体来说，Hessian矩阵的稀疏性与SLAM的markov random field的邻接矩阵稀疏性相同(严格讲邻接矩阵对角元素为0，因此是与Lapacian矩阵相同)。马尔可夫随机场是一种无向图概率图模型，且只包含变量节点，若连接一条边表示两个节点之间具有某种联系，类似g2o中图优化的定义。
<div align="center"><img src="../resources/chapter3/hessian_sparsity.png" width=50%></div>

## 三、(因子图)消元算法
(因子图)消元算法是把**因子图转换成贝叶斯网络的算法**。

<div align="center"><img src="../resources/chapter3/elimination_algorithm.png" width=50%></div>

具体应用如下所示，每次删除一个红色变量节点周边对应因子，并构建一个新的周边因子。消元把因子图一个变量节点变成贝叶斯网络的结果节点。

<div align="center"><img src="../resources/chapter3/elimination_algorithm_application.png" width=50%></div>

综上，实际上对应如下分解：
$$\mathbb{P}(l_1, l_2, x_1, x_2, x_3) = \mathbb{P}(l_1|x_1, x_2)\mathbb{P}(l_2|x_3)\mathbb{P}(x_1|x_2)\mathbb{P}(x_2|x_3)\mathbb{P}(x_3)$$

## 四、稀疏矩阵QR分解与(因子图)消元
这里承接第三节，说明(因子图)消元算法与稀疏矩阵QR分解的**等价性**。
这里用因子图对应的Jacobi矩阵来说明，则第三节对应的消去变量$l_1$，等价于把Jacobian第一列的所有非零元素拼成一个大矩阵$[\bar{A_1}, \bar{b_1}]$，然后使用QR分解，得
<div align="center"><img src="../resources/chapter3/QR.png" width=50%></div>

具体例子如下所示，上图对应的是消去$l_1$的过程，下图对应把原本的雅各比矩阵进行分解的过程，其中R是上三角矩阵。蓝色是贝叶斯网络连的边，红色是新的因子。
<div align="center"><img src="../resources/chapter3/elimination_factorization.png" width=50%></div>

整个因子图消元过程等价于雅各比矩阵稀疏QR分解过程。
<div align="center"><img src="../resources/chapter3/elimination_factorization_full.png" width=50%></div>

在通过QR分解得到上三角矩阵之后，通过回代即可求解线性系统$[A|b]$。

## 五、总结
不同矩阵的稀疏结构与概率图模型对应关系如下所示：

1. 上三角矩阵-贝叶斯网络(有向无环图DAG)
2. Jacobian矩阵-因子图
3. Hessian矩阵-马尔可夫随机场

综上，本章揭露了矩阵分解、(因子图)消元与概率图模型之间的关系。第三节说明了因子图消元本质上等价于因子图模型转换为贝叶斯网络模型；第四节说明了因子图消元等价于稀疏矩阵QR分解。
$$因子图消元 \leftrightarrow 概率图模型转换 \leftrightarrow 稀疏矩阵QR分解$$
QR分解定义如下
$$A(m\times n) = Q(m\times n) R(n\times n)$$
这给了一个不同的视角，即QR分解是把一个因子图A(m个因子和n个变量)变成了一个Bayes网络R(n个节点)。

## 参考资料
[1] [知乎-图的拉普拉斯矩阵](https://zhuanlan.zhihu.com/p/362416124)